In [ ]:
print('script starts')

# Import necessary libraries
import psutil  # Used in print_memory_usage function
import openai  # Used in get_completion function and main function
from tqdm import tqdm  # Used in main function for progress bar
import time  # Used in main function for sleep
#from langchain.embeddings.openai import OpenAIEmbeddings  # Used in main function
from langchain_openai import OpenAIEmbeddings
#from langchain_community.vectorstores import Chroma  # Used in main function
from langchain_chroma import Chroma
import os  # Used in main function
import pandas as pd
import re

# Function to print memory usage
def print_memory_usage():
    process = psutil.Process()
    mem_info = process.memory_info()
    print(f"Memory Usage: {mem_info.rss / (1024 ** 2)} MB")

# Function to get completion from OpenAI
def get_completion(prompt, model="gpt-4o"):
    try:
        response = openai.ChatCompletion.create(
            model=model,
            messages=[{"role": "user", "content": prompt}],
            temperature=0
        )
        return response.choices[0].message.content
    except Exception as e:
        print(f"Error in get_completion: {e}")
        return None

print('Functions and libraries are loaded')
print_memory_usage()

# Function to get completion from OpenAI
def get_completion(prompt, model="gpt-4o"):
    response = openai.ChatCompletion.create(
        model=model,
        messages=[{"role": "user", "content": prompt}],
        temperature=0
    )
    return response.choices[0].message.content

print('Functions and libraries are loaded')
print_memory_usage()

# Set OpenAI API key
OPENAI_API_KEY = 'your-api-key-here'  # Replace with your actual API key
openai.api_key = OPENAI_API_KEY


# Set up the embedding and vector database
persist_directory = 'docs Light in French Literature/chroma/'
embedding = OpenAIEmbeddings(openai_api_key=OPENAI_API_KEY)
vectordb = Chroma(persist_directory=persist_directory, embedding_function=embedding)

print('Vector database is loaded')
print_memory_usage()

# Antal af dokumenter i samlingen
print('collection size')
print(vectordb._collection.count())

# Decide the number of output results
K = 10
fetch_k = K + 10

# Define the question and perform similarity search

question = "Context: these texts sometimes mention artificial lighting, such as lamps, candles, streetlamps, and so on. \
            Please find examples where illumination by artificial lighting contributes to romantic or loving emotions."


print('Searching starts ... ')
# Simulate the fetching process with a progress bar

for i in tqdm(range(fetch_k), desc="Fetching documents"):
    # Simulate a delay for each fetch
    time.sleep(0.1)  # Replace this with the actual fetching logic if possible

# Now perform the max marginal relevance search
print('perform the max marginal relevance search')
try:
    docs_mmr = vectordb.max_marginal_relevance_search(question, k=K, fetch_k=fetch_k)
    print('Search results stored in docs variable')
except Exception as e:
    print(f"Error in max_marginal_relevance_search: {e}")
    docs_mmr = []


# Two empyt lists for storing the search output 
langchain_mmr_results_sources = []
langchain_mmr_results_content = []


# Process and store the mmr results
for count, doc in enumerate(docs_mmr, start=1):
    content = doc.page_content
    content = (content.replace('\n', ' ').replace('Ãˆ', 'È').replace('Ãª', 'ê').replace('â€™', '’')
                      .replace('Å“', 'œ').replace('Ã©', 'é').replace('Ã®', 'î')
                      .replace('Ã‰', 'É').replace('Ã¨', 'è').replace('Ã', 'à')
                      .replace('à¢', 'â').replace('â€¦', '…').replace('à”', 'Ô')
                      .replace('à¹', 'ù'))

    source = doc.metadata.get('source', '').replace('}', '').replace('.txt', '').split('\\')[-1]
    print (source)
    print (content)
    print (10 * '*')

    # store output
    langchain_mmr_results_sources.append(source)
    langchain_mmr_results_content.append(content)
    
print('Clean up variables that are no longer needed')
# Clean up variables that are no longer needed
del docs_mmr, doc, content, source

print('Clean up variables that are no longer needed')
print_memory_usage()


print('Saving the max marginal relevance search result as csv')
df = pd.DataFrame({'source': langchain_mmr_results_sources, 'content':langchain_mmr_results_content})
print (df)
# save langchain search result
df.to_csv('langchain_mmr_search_result.csv', index=False)


print('Clean up variables that are no longer needed')
# Clean up remaining variables
del df, langchain_mmr_results_sources, langchain_mmr_results_content, fetch_k
print_memory_usage()

print('MMR script done!')


print ('sim script starting!')

# Two empyt lists for storing the similarity search output 
langchain_sim_results_sources = []
langchain_sim_results_content = []

# Process and store the similarity results
docs = vectordb.similarity_search(question,k=K)
for doc in docs:
    content = doc.page_content
    content = (content.replace('\n', ' ').replace('Ãˆ', 'È').replace('Ãª', 'ê').replace('â€™', '’')
                      .replace('Å“', 'œ').replace('Ã©', 'é').replace('Ã®', 'î')
                      .replace('Ã‰', 'É').replace('Ã¨', 'è').replace('Ã', 'à')
                      .replace('à¢', 'â').replace('â€¦', '…').replace('à”', 'Ô')
                      .replace('à¹', 'ù'))
    
    source = doc.metadata.get('source', '').replace('}', '').replace('.txt', '').split('\\')[-1]
    print (source)
    print (content)
    print (10 * '*')

    # store output
    langchain_sim_results_sources.append(source)
    langchain_sim_results_content.append(content)

print('Clean up variables that are no longer needed')
# Clean up variables that are no longer needed
del docs, doc, content, source

print('Saving the similarity search result as csv')
df = pd.DataFrame({'source': langchain_sim_results_sources, 'content':langchain_sim_results_content})
print (df)
# save langchain search result
df.to_csv('langchain_sim_search_result.csv', index=False)

print('Sim script done!')


print ('Script done.')
